In [1]:
# Imports
import cv2
import time
import csv
import serial
import math
import random
import struct
from threading import Thread

Starting Serial Thread...
Serial Thread running
Camera Running...
got label 0.3
Saving img 1560770921.3120744_0.3.png
attenpt 1 successful!
got label 0.58
Saving img 1560770922.561371_0.58.png
attenpt 2 successful!
got label 0.84
Saving img 1560770923.8052335_0.84.png
attenpt 3 successful!
got label 1.12
Saving img 1560770925.0589235_1.12.png
attenpt 4 successful!
got label 1.41
Saving img 1560770926.3050992_1.41.png
attenpt 5 successful!
5 out of 5 attempts successful
closing Threads...


In [ ]:
#Class and Function Definitions

class Ser(Thread):
    def __init__(self, port):
        super().__init__()
        self._run = True
        self._ser = serial.Serial(port ,115200)
        self._last_value = 0.0
        self._valid = False

    def get_act_value(self):
        return self._last_value
    
    def value_valid(self):
        return self._valid
    
    def write(self, random_rad):
        self._ser.write(str(random_rad).encode('utf-8'))

    def run(self):
        print('Serial Thread running')
        while self._run:
            line = self._ser.read_until()
            #default reading: "b'1, 1.18/r/n"
            self._valid = bool(str(line).split(", ")[0].split("'")[1])
            self._last_value = float(str(line).split(", ")[1].split("\\")[0])

    def end(self):
        self._run = False
        self.join()
        self._ser.close()


class PsEye(Thread):
    def __init__(self):
        super().__init__()
        self._run = True
        self._video = cv2.VideoCapture(0)
        self._last_frame = None

    def get_act_frame(self):
        return self._last_frame

    def run(self):
        while self._run:
            check, frame = self._video.read()
            if check:
                self._last_frame = frame

    def save_last_frame(self, img_file_path, img_name):
        print('Saving img {}'.format(img_name))
        cv2.imwrite(str(img_file_path + img_name), self._last_frame)

    def end(self):
        self._run = False
        self.join()
        self._video.release()

    def frame_valid(self):
        return self._last_frame is not None

################################################################
#Main Functions
def get_label_value():
    for attempt in range(10):
        if ser.value_valid():
            repeat = False
            label = ser.get_act_value()
            print("got label {}".format(str(label)))
            return (True, label)
        else:
            print("No valid value")
        time.sleep(0.001)
    return (False, None)
     
    
def log_csv(file, img_name, label):
    with open(file, 'a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow([img_name, label])
    pass

In [3]:
#Available Ports:
!python -m serial.tools.list_ports

/dev/ttyACM0        
/dev/ttyACM1        
2 ports found


In [ ]:
#Main Programm

port = '/dev/ttyACM1'

csv_file_path = "./image_labels.csv"
img_file_path = "./pictures/"

num_images = 5


print("Starting Serial Thread...")
ser = Ser(port)
ser.start()
time.sleep(1)

print("\nStarting Camera Thread...")
camera = PsEye()
camera.start()

# Waiting until camera ready
while not camera.frame_valid():
    time.sleep(0.05)

print('Camera Thread running...')

img_counter = 0

for a in range(num_images):
    rand_increment = random.randint(100,200) 
    rand_rad = rand_increment/4096 * (2*math.pi)
    ser.write(rand_rad)
    time.sleep(1.2)
    (success, label) = get_label_value()
    if success:
        label =label %(2*math.pi)
        img_name = "{}_{}.png".format(time.time(), label)
        camera.save_last_frame(img_file_path, img_name)
        log_csv(csv_file_path, img_name, label)
        img_counter += 1
        print('attenpt {} successful!'.format(a+1))
    else:
        print('attenpt {} FAILED!'.format(a+1))
        
print("{} out of {} attempts successful".format(img_counter, num_images))
camera.end()
ser.end()
print("closing Threads...")